# 2016 and 2020 presidential elections in the Bronx

Here, we will be looking at 2016 and 2020 election night results to see how Bronxites voted in the previous elections. The objectives are to visualize the data and identify trends, if any.

Note: The 2020 dataset was problematic — the first 10 columns were just column names and the next 10 columns did not have headers. I corrected the dataset manually.

In [1]:
# importing libraries
import pandas as pd
import os

In [2]:
# reading files

df_2016 = pd.read_csv("data/2016-Citywide President Vice President Citywide EDLevel.csv")
df_2020 = pd.read_csv("data/2020-Citywide President Vice President Citywide EDLevel.csv")

## Filtering data by county

We want to keep only data for the Bronx, or assembly districts 77 to 87.

In [3]:
def filter_county(df, col_name, county):
    """
    This function filters a DataFrame to keep only a specific county.

    Parameters:
    df (pd.DataFrame) = name of DataFrame
    col_name (str) = name of column
    county (str) = name of county to filter
    """

    filtered_df = df[df[col_name] == county]
    return filtered_df

In [4]:
# filtering only bronx data
bx_2016 = filter_county(df_2016, "County", "Bronx")
bx_2020 = filter_county(df_2020, "County", "Bronx")

## Pulling Assembly District and Election District codes

We will create a new column that shows both the assembly district and election district numbers, which is the same code we will use for visualization. This follows the format of our map and geocodes for viz, i.e., AE0ED.

In [5]:
def add_ed(df, new_col, ad_col, ed_col):
    """
    This function gets only the numbers of the Assembly Districts and Election Districts and combines them.
    The resulting new column will show a five-digit code for mapping. 

    This function presupposes that items in the Assembly Districts and Election Districts are integers.

    Parameters:
    df (pd.DataFrame) = name of DataFrame
    new_col (str) = name of new column that will hold the five-digit code that combines the Assembly District and Election District numbers
    ad_col (str) = column name of the Assembly District number
    ed_col (str) = column name of the Election District number
    """

    df.loc[:, new_col] = df[ad_col].astype(str) + df[ed_col].astype(str).str.zfill(3)
    return df

In [6]:
# create a new column that combines AD and ED digits
bx_2016 = add_ed(bx_2016, "ed_map_for_viz", "AD", "ED")
bx_2020 = add_ed(bx_2020, "ed_map_for_viz", "AD", "ED")

/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_14690/3299567341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, new_col] = df[ad_col].astype(str) + df[ed_col].astype(str).str.zfill(3)
/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_14690/3299567341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, new_col] = df[ad_col].astype(str) + df[ed_col].astype(str).str.zfill(3)


## Filter candidate

Here, we will create a new DataFrame that filters only the candidates we want.

In [7]:
def filter_candidates(df, col_name, candidate_name, status_col="EDAD Status"):
    """
    This function filters a DataFrame in two ways: a) keep only certain candidates,
    b) include rows of districts that were "combined" with other districts.

    Parameters:
    df (pd.DataFrame) = name of DataFrame
    col_name (str) = name of column
    candidate_name (str) = name of candidate/s to filter; use the pipe symbol ("|") for multiple names
    status_col (str): name of status column, defaults to "EDAD Status"
    """

    # filter candidates
    filtered_candidates = df[df[col_name].str.contains(candidate_name, case=False, na=False)]

    # filter combined rows
    filtered_combined = df[df[status_col].str.contains("combined", case=False, na=False)]

    # concat both dfs and remove duplicates, then sort to order
    filtered_df = pd.concat([filtered_candidates, filtered_combined]).drop_duplicates()
    filtered_df = filtered_df.sort_index()
    
    return filtered_df
    

In [8]:
# keeping only democrat and republican votes data

# 2016: Trump vs Clinton
bx_2016_prez = filter_candidates(bx_2016, "Unit Name", "Trump|Clinton")

# 2020: Trump vs Biden
bx_2020_prez = filter_candidates(bx_2020, "Unit Name", "Trump|Biden")

## Getting total votes per candidate

Update: An earlier version of the code calculated only vote counts, but because we will be comparing results with other years, we will use percentages for consistency.

In [9]:
def process_votes(df, col_name, candidate_names, tally_col, ed_col, new_col, status_col="EDAD Status"):
    """
    This function counts the votes per candidate per election district.
    It will include combined districts.

    Parameters:
    df (pd.DataFrame) = name of DataFrame
    col_name (str) = column name to search for candidate names
    candidate_names (list) = list of candidates to filter; this assumes a two-party race
    tally_col (str) = column name for vote counts
    ed_col (str) = column name for election districts
    new_col (str) = name of new column that holds vote differential value
    status_col (str): name of status column, defaults to "EDAD Status"
    """

    # filters the candidates and sums votes
    results = {} # holds our data

    # makes sure we're working with int values
    df[tally_col] = df[tally_col].astype(int)
    
    for candidate in candidate_names:
        candidate_df = df[
            (df[col_name].str.contains(candidate, case=False, na=False)) |
            (df[status_col].str.contains("combined", case=False, na=False))
        ]

        results[candidate] = candidate_df.groupby(ed_col)[tally_col].sum()

    # saves to df
    result_df = pd.DataFrame(results).reset_index().fillna(0)
    result_df[candidate_names] = result_df[candidate_names].apply(pd.to_numeric, errors="coerce")

    # computes vote differences
    result_df[new_col] = result_df[candidate_names[0]] - result_df[candidate_names[1]]

    return result_df

In [10]:
totals_2016 = process_votes(bx_2016_prez, "Unit Name", ["Trump", "Clinton"],\
                      "Tally", "ed_map_for_viz",\
                      "vote_differences")
totals_2016

,ed_map_for_viz,Trump,Clinton,vote_differences
0,77001,10,200,-190
1,77002,11,219,-208
2,77003,10,378,-368
3,77004,19,464,-445
4,77005,20,702,-682
...,...,...,...,...
954,87091,0,0,0
955,87092,0,0,0
956,87093,0,0,0
957,87094,0,0,0


In [11]:
totals_2020 = process_votes(bx_2020_prez, "Unit Name", ["Trump", "Biden"],\
                      "Tally", "ed_map_for_viz",\
                      "vote_differences")
totals_2020

,ed_map_for_viz,Trump,Biden,vote_differences
0,77001,27,199,-172
1,77002,20,202,-182
2,77003,84,515,-431
3,77004,87,408,-321
4,77005,70,439,-369
...,...,...,...,...
994,87098,68,500,-432
995,87099,62,458,-396
996,87100,30,331,-301
997,87101,67,415,-348


In [12]:
def total_votes(df, col1, col2):
    """
    This function totals the values per group in a column.

    Parameters:
    df (pd.DataFrame) = name of DataFrame
    col1 (str) = name of column by which we will group values
    col2 (str) = name of column that holds the values to add
    """
    # make sure that the column values are integers
    df[col2] = df[col2].astype(int)

    # creates a lean DataFrame that holds total values per group
    new_df = df.groupby(col1)[col2].sum().reset_index()
    new_df.rename(columns = {col2: "total_value"}, inplace=True)
    
    return new_df

In [13]:
bx_2016_total = total_votes(bx_2016, "ed_map_for_viz", "Tally")
bx_2020_total = total_votes(bx_2020, "ed_map_for_viz", "Tally")

/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_14690/1404803246.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col2] = df[col2].astype(int)
/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_14690/1404803246.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col2] = df[col2].astype(int)


In [14]:
# merging our datasets

totals_2016 = pd.merge(totals_2016, bx_2016_total, on="ed_map_for_viz")
totals_2020 = pd.merge(totals_2020, bx_2020_total, on="ed_map_for_viz")

## Get percentage of votes per candidate

In [15]:
def get_pct(df, target_col, total_col, new_col):
    """
    This function calculates the percentage of a column value.

    Parameters:
    df (pd.DataFrame) = name of DataFrame
    target_col (str) = name of column that contains the dividend
    total_col (str) = name of column that contains the divisor
    new_col (str) = name of new column that holds percentage values
    """
    
    # make sure that the column values are integers
    df[target_col] = df[target_col].astype(int)
    df[total_col] = df[total_col].astype(int)

    # getting percentage
    df[new_col] = (df[target_col] / df[total_col] * 100)
    df.fillna({new_col: 0}, inplace=True)
    
    return df
    

In [16]:
# getting percentages per candidate

totals_2016 = get_pct(totals_2016, "Trump", "total_value", "trump_pct")
totals_2016 = get_pct(totals_2016, "Clinton", "total_value", "clinton_pct")

totals_2020 = get_pct(totals_2020, "Trump", "total_value", "trump_pct")
totals_2020 = get_pct(totals_2020, "Biden", "total_value", "biden_pct")

In [17]:
# get vote differential in percentage points

totals_2016["vote_counts_pct"] = totals_2016["trump_pct"] - totals_2016["clinton_pct"]
totals_2020["vote_counts_pct"] = totals_2020["trump_pct"] - totals_2020["biden_pct"]

In [18]:
# save to csv
os.makedirs("csv", exist_ok=True)

totals_2016.to_csv("csv/2016-votes.csv", encoding="UTF-8", index=False)
totals_2020.to_csv("csv/2020-votes.csv", encoding="UTF-8", index=False)

### Old version

The lines of code above results in datasets with only the columns we need: election district, candites, and vote differential. 

Here's the previous processing I did, which breaks down the steps and adds columns to the main dataset.

In [19]:
# def consol_candidate(df, col_name, candidate_name):
#     """
#     This function consolidates candidates per election district. 

#     Parameters:
#     df (pd.DataFrame) = Pandas DataFrame
#     col_name (str) = name of the column to query for the keyword or candidate name search
#     candidate_name (str) = string for keyword or candidate name search
#     """

#     filtered_df = df[df[col_name].str.contains(candidate_name, case=False, na=False)]
#     return filtered_df

In [20]:
# col_name = "Unit Name"

# # creating dfs per candidate per election year
# bx_trump_2016 = consol_candidate(bx_2016_prez, col_name, "Trump")
# bx_clinton_2016 = consol_candidate(bx_2016_prez, col_name, "Clinton")
# bx_trump_2020 = consol_candidate(bx_2020_prez, col_name, "Trump")
# bx_biden_2020 = consol_candidate(bx_2020_prez, col_name, "Biden")

In [21]:
# def count_votes(df, col_name, col_value):
#     """
#     This function totals the number of votes per candidate. 

#     Parameters:
#     df (pd.DataFrame) = Pandas DataFrame
#     col_name (str) = name of the column that anchors the data
#     col_value (str) = string for keyword or candidate name search
#     """

#     totals_df = df.groupby(col_name)[col_value].sum().reset_index()
#     return totals_df

In [22]:
# col1 = "ed_map_for_viz"
# col2 = "Tally"

# # summing votes per candidate per election year per election district
# bx_trump_2016 = count_votes(bx_trump_2016, col1, col2)
# bx_clinton_2016 = count_votes(bx_clinton_2016, col1, col2)
# bx_trump_2020 = count_votes(bx_trump_2020, col1, col2)
# bx_biden_2020 = count_votes(bx_biden_2020, col1, col2)

In [23]:
# # renaming columns to prepare for merging

# bx_trump_2016.rename(columns={col2: "Trump"}, inplace=True)
# bx_clinton_2016.rename(columns={col2: "Clinton"}, inplace=True)
# bx_trump_2020.rename(columns={col2: "Trump"}, inplace=True)
# bx_biden_2020.rename(columns={col2: "Biden"}, inplace=True)

In [24]:
# # merging dfs per election year

# totals_2016 = pd.merge(bx_trump_2016, bx_clinton_2016, on="ed_map_for_viz")
# totals_2020 = pd.merge(bx_trump_2020, bx_biden_2020, on="ed_map_for_viz")

In [25]:
# # convert our columns to int

# totals_2016[["Trump", "Clinton"]] = totals_2016[["Trump", "Clinton"]].astype(int)
# totals_2020[["Trump", "Biden"]] = totals_2020[["Trump", "Biden"]].astype(int)

In [26]:
# # getting the difference of votes
# # for consistency, Dems will have a negative value

# totals_2016["votes_value_for_viz"] = totals_2016["Trump"] - totals_2016["Clinton"]
# totals_2020["votes_value_for_viz"] = totals_2020["Trump"] - totals_2020["Biden"]

In [27]:
# # save to csv

# totals_2016.to_csv("2016-votes.csv", encoding="UTF-8", index=False)
# totals_2020.to_csv("2020-votes.csv", encoding="UTF-8", index=False)